<a href="https://colab.research.google.com/github/andersknudby/Remote-Sensing/blob/master/GEE%20Lab%203.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Lab 3: MODIS Time Series and Graphing

## Introduction

In this lab we will continue working with the Google Earth Engine python API in Google Colab. We will explore the available MODIS datasets and create a sea-surface temperature (SST) time series with MODIS. We will also explore some python graphing libraries to create interactive plots of earth engine data.


**Files you need:** N/A

**Preparation:**

## Steps

###| 1. Imports, installation, authentication, initialization

Our first step is to get the notebook set up. Review the description in the first lab if you want a refresher on why we need to do this!


In [ ]:
# Install geemap library so that we can use it to view images on an interactive map
## You may get a runtime warning - you can ignore that
!pip install geemap

In [ ]:
# Import the necessary libraries
import ee
import numpy as np
import geemap as geemap
import pprint

import pandas as pd
import altair as alt
import folium

# Set up a 'pretty printer' to print ...
pp = pprint.PrettyPrinter(depth=3)

In [ ]:
# Authenticate and initialize this instance of GEE in Google Colab
## Follow the prompts and fill in authentication code
ee.Authenticate()
ee.Initialize()

### | 2. MODIS

So far in these GEE labs, we have almost exclusively used Landsat data. Landsat data is good for learning GEE because it has well-known TOA and surface reflectance products that cover a very long time period, has a good spectral resolution,  and has decent spatial resolution for many applications. Because of this, there is also a lot of supporting documentation both by Google and by other users of GEE for using Landsat imagery in GEE. 

However, one of the main benefits of using GEE is how easy it is to access huge amounts of *different kinds of data*. GEE combines whole collections of imagery and imagery products from a variety of satellites and organizations  and makes it easy to analyze them, without having to download individual images from the source.

So, to better understand the power of Google Earth Engine, and to expose you to some more code you may be able to use in the future, for this lab we will take a look at moderate spatial resolution, high temporal resolution data with MODIS.





#### MODIS Info

MODIS (or Moderate Resolution Imaging Spectroradiometer) is an instrument aboard two complimentary satellites launched by NASA: **Terra** and **Aqua**. Combined, the Terra and Aqua MODIS instruments image the entire surface of the Earth every 1 to 2 days, acquiring data in 36 spectral bands at 250 m (bands 1–2), 500 m (bands 3–7), and 1000 m (bands 8–36) resolution. 
 
With its low-ish spatial resolution but high temporal resolution, MODIS data are useful to track changes in the landscape over time.

Like Landsat, MODIS has data products in addition to the 36 band imagery that you can access from the GEE catalogue. Check out all the available data in the [MODIS section of the GEE catalogue](https://developers.google.com/earth-engine/datasets/catalog/modis).

The structure of MODIS data is a bit more complicated than what we have dealt with so far. The data from MODIS is processed by a variety of groups (depending on the product) in order to create the available data products. Unlike some other satellite data, MODIS does not provide unprocessed scenes to the general public. 




#### Using MODIS Surface Reflectance
To start, we will use the MOD09GA.006 Terra Surface Reflectance Daily Global 1km and 500m daily data product. This collection has surface reflectance available at 500m resolution, as well as pixel quality assurance (QA) bit information at the 1000m resolution. These QA bits contain information about cloud contamination, as well as the number of observations that make up a given pixel -- remember, these pixels are data ***products*** and so can represent >1 observation.

Masking unwanted pixels is done in a similar way as for Landsat, using the MODIS pixel QA bands. In this case we will create functions to mask both cloud pixels (from the `state_1km` QA band) and pixels with no observations (from the `num_observation_1km` band). 



In [ ]:
#Calculate how frequently a location is labeled as clear (i.e. non-cloudy)
#according to the "internal cloud algorithm flag" of the MODIS "state 1km"
#QA band.

# A function to mask out pixels that did not have observations.
def maskEmptyPixels(image):
  # Find pixels with >0 observations and mask out all other pixels 
  withObs = image.select('num_observations_1km').gt(0)
  return image.updateMask(withObs)

# A function to mask out cloudy pixels.
def maskClouds(image):
  #Select the QA band.
  QA = image.select('state_1km')
  # Make a mask to get bit 10, the internal_cloud_algorithm_flag bit.
  bitMask = 1 << 10
  # Return an image masking out cloudy areas.
  return image.updateMask(QA.bitwiseAnd(bitMask).eq(0))


Now let's apply the masking functions to a collection and display it on the map

In [ ]:
# Start with an image collection for summer 2010 and apply masks
summer2010 = ee.ImageCollection('MODIS/006/MOD09GA')\
        .filterDate('2010-04-01', '2010-09-30')\
        .map(maskEmptyPixels)\
        .map(maskClouds)

# note that slashes \ are used in the code above to allow another line of code for 
# formatting appearance (the code could all be on one long line, if you wanted)
# in some cases, code automatically continues to the next line, e.g. when there are commas,
# but sometimes we need to tell the code that we are continuing on the next line with a \

# Get the median pixel value for summer 2010
# Recall from the last lab that this is a temporal reducer
summer2010med = summer2010.median()

# Note that true colour for this MODIS dataset is Band 1 in Red, Band 4 in Green, and Band 3 in Blue
visParams = {'bands': ['sur_refl_b01', 'sur_refl_b04', 'sur_refl_b03'],
     'gain': 0.07,
     'gamma': 1.4
    }

Map = geemap.Map()
Map.addLayer(summer2010med,visParams,'Summer 2010 Median SR')
Map.addLayerControl()
Map.setCenter(41.92,32.36,5)
Map

Notice that we did not filter the bounds of the image so we have created a median for the entire globe (where there is data for summer 2010). So you can pan and zoom to any area you like and see what the data look like there.

Because of the high temporal frequency of MODIS, the resulting imagery is pretty great, because the median pixel is potentially calculated from as many as 182 pixels. That is, one pixel per day for every day from April 1st to September 30th (182 days).


#### Using other MODIS data products

In addition to the surface reflectance products, MODIS also provides special data products calculated from the data. For the next example, we will use the Ocean Color Standard Mapped Image MODIS Aqua Data. 

This dataset is a level 3 product that includes ocean color and satellite ocean biology data produced by NASA's Earth Science Data Systems (ESDS) program. It includes bands that are useful for calculating ocean biology data because of their narrow wavelengths, as well as some bands of pre-calculated data such as chlorophyll-a concentration and sea-surface temperature (SST).

Let's check out the SST band and use it to look at El Nino and La Nina years. If you need a quick refresher, check out the NASA kid science pages for [El Nino](https://spaceplace.nasa.gov/el-nino/en/) and [La Nina](https://spaceplace.nasa.gov/la-nina/en/).

In [ ]:
# Load MODIS Ocean Colour Image Collection
modisOceanColor = ee.ImageCollection('NASA/OCEANDATA/MODIS-Aqua/L3SMI');

# Create a collection from the SST (sea surface temperature) band
sst = modisOceanColor.select(['sst'])

# Calculate median SST values for winter 2015/2016 - an El Nino year
sstElNino = sst.filterDate('2015-12-01', '2016-02-28').median()

# Calculate median SST values for winter 2010/2011 - a La Nina year
sstLaNina = sst.filterDate('2010-12-01', '2011-02-28').median()


In [ ]:
# Create vis parameters for SST with a palette that goes from blue to red
vis = {'min': -4, 'max': 35, 'palette': '#2166AC,#4393C3,#92C5DE,#D1E5F0,#B2182B'};

Map = geemap.Map()
Map.addLayer(sstElNino,vis,'El Nino')
Map.addLayer(sstLaNina,vis,'La Nina')
# set the map centre as the middle of the Pacific at a zoom level of two (pretty zoomed out)
Map.setCenter(-146.25,0,2)
Map.addLayerControl()
Map

Toggle the layers on and off to compare the sea surface temperature for El Nino and La Nina years. 

Recall from your climatology courses (or the NASA Kids pages linked above) that El Nino results from weaker trade winds moving from East to West resulting in warmer waters toward the west coast of Central/South America. La Nina occurs in years with stronger than normal trade winds, resulting in cooler waters off the west coast of Central/South America.

The effect of the El Niño-Southern Oscillation (ENSO) is best visualized as a temperature anomaly in order to show the effect on areas that still have relatively cool waters, rather than an absolute temperature which shows 10-degree water as light blue, regardless of whether 10 degrees is colder or warmer than normal conditions. However, these two layers still do a decent job of showing how MODIS can be used to look at large scale data patterns. 

### | 3. Time Series Charts with MODIS


Time series analysis involves looking at data over time to detect and describe change. With imagery this can be done in the form of a video, or, more simply, by extracting some data from the imagery and displaying the change through time on a graph.

The Google Earth Engine Code Editor has a built in charting library called Chart.UI that integrates some earth engine functionality like reducers into chart functions to create easy data visualizations.

Unfortunately, Chart.UI does not work in the python API, so we need to create some reducing functions that we can apply to the image collections before plotting with some normal python charting libraries. We will look at two charting libraries in this lab: the simple **matplotlib**, and **Altair**, which is a library that is useful for creating nice interactive maps.

#### Create Reducing Function

In order to create a time series analysis, we can either look at the change in pixel values over time for a **single pixel** (at a defined point), or we can look at the change in values over time for a **region of pixels** (within a defined geometry). In both cases, we need to use a reducer to summarize the values for an area before plotting the time series. 

As mentioned, the Chart.UI library incorporates this reducing capability into its chart functions, but in the Python API in colab, we need to define the functions that will apply a reducer across an image collection before we plot the time series. The functions below were taken directly from a [GEE Python Charting Tutorial ](https://colab.research.google.com/github/google/earthengine-community/blob/master/tutorials/time-series-visualization-with-altair/index.ipynb). 

The first function is actually used to create a function in a nifty bit of python. The code below defines a reusable function that can perform the reducing task for different datasets. The function accepts arguments such as scale and reduction method to parameterize the operation for each particular analysis.

Do not worry too much about trying to understand *how* the functions work. For our purposes, we just need to know that they work (they do) and that we can re-use them to convert image collections to tables.


In [ ]:
# Define a function (and embedded function) for reducing a region for charting
# See the function descriptions defined in red within the functions

def create_reduce_region_function(geometry,
                                  reducer=ee.Reducer.median(),
                                  scale=500,
                                  crs='EPSG:4326',
                                  bestEffort=True,
                                  maxPixels=1e13,
                                  tileScale=4):
  """Creates a region reduction function.

  Creates a region reduction function intended to be used as the input function
  to ee.ImageCollection.map() for reducing pixels intersecting a provided region
  to a statistic for each image in a collection. See ee.Image.reduceRegion()
  documentation for more details.

  """

  def reduce_region_function(img):
    """
      Function returns
      An ee.Feature that contains properties representing the image region
      reduction results per band and the image timestamp formatted as
      milliseconds from Unix epoch (included to enable time series plotting).
    """

    stat = img.reduceRegion(
        reducer=reducer,
        geometry=geometry,
        scale=scale,
        crs=crs,
        bestEffort=bestEffort,
        maxPixels=maxPixels,
        tileScale=tileScale)

    return ee.Feature(geometry, stat).set({'millis': img.date().millis()})
  return reduce_region_function


Now that we have defined our functions, we can use them on an image collection and a region in order to get a table we can use as input to some charting library functions.

In [ ]:
# Load MODIS Ocean Colour Image Collection
modisOceanColor = ee.ImageCollection('NASA/OCEANDATA/MODIS-Aqua/L3SMI');

# Create a collection from the SST (sea surface temperature) band
sst = modisOceanColor.select(['sst'])

# Define an area of interest (aoi)
# We will look at Lake Erie 
aoi = ee.Geometry.Polygon([[-83.594,41.326],
                           [-78.782,41.326],
                           [-78.782,42.972],
                           [-83.594,42.972],
                           [-83.594,41.326]])


In [ ]:
# Check out the AOI if you want
# If you know where Lake Erie is, feel free to skip this cell
Map = geemap.Map()
Map.addLayer(aoi,{},'AOI')
Map

The next code chunk does three things:
1. Assigns the reduce function with specific parameters to the function, `reduce_sst`. This will make it simpler to map the function across the image collection.
2. Maps the function over the `sst` image collection to reduce each image.
3. Filters out any resulting features that have null computed values (occurs when all pixels in an AOI are masked).

In [ ]:
# Use the above defined function to create a reducing function specific to the SST dataset
## We are going to leave most of the arguments as their defaults (see the above code)
## except geometry which we set to our aoi, the reducer which we will set as the median
reduce_sst = create_reduce_region_function(
    geometry=aoi, reducer=ee.Reducer.median())

# Convert the image collection to a feature collection with the reducer function
## this step just applies the function we created above (reduce_sst), with its defined inputs to our
## image collection
sst_stat_fc = ee.FeatureCollection(sst.map(reduce_sst))

# Filter out null values based on bandnames of the first image in the feature collection
sst_stat_fc = sst_stat_fc.filter(
    ee.Filter.notNull(sst.first().bandNames()))

The next step to format the data for charting is to create an `fc_to_dict` function to convert the `ee.FeatureCollection` to `ee.Dictionary`. Then we convert the `ee.Dictionary` to a pandas dataframe, which is a common table format in python.

Recall that Google Earth Engine works by sending code between your computer in the python API and Google's servers for computation. In order to plot charts without Google's built in Chart.UI functioning, we need to transfer the data from Google's server (often referred to in documentation as 'server side') to our own computers (referred to as 'client side'), which we do with the `.getInfo()` call. 

This workflow is very useful for integrating Google Earth Engine data with other python libraries and data processing functions, not just for charting. 

But now back to charting...

In [ ]:
# Define a function to transfer feature properties to a dictionary.
# We use this to convert our earth engine object (reduced image collection) to a 
# collection of properties and their values 

def fc_to_dict(fc):
  prop_names = fc.first().propertyNames()
  prop_lists = fc.reduceColumns(
      reducer=ee.Reducer.toList().repeat(prop_names.size()),
      selectors=prop_names).get('list')

  return ee.Dictionary.fromLists(prop_names, prop_lists)

Now we need to get the data, in the form of a dictionary, on to our own computer:

In [ ]:
# Convert feature collection to dictionary and use .getInfo() to transfer to our own computers
sst_dict = fc_to_dict(sst_stat_fc).getInfo()


Now let's see what the dictionary looks like.

In [ ]:
# print normally
print(sst_dict,"\n") # \n tells it to print a new line

# print some of the dictionary with formatting
for prop in sst_dict.keys():
    print(prop + ':', sst_dict[prop][0:3] + ['...'])

In the dictionary format, lists of data are stored in properties in the format `{property1: [list of values1],property2: [list of values2]...}`. This can easily be converted to a dataframe by creating columns from the properties. To do that we will use the .DataFrame() function from the pandas library (which we imported as pd).


In [ ]:
# Convert the dictionary to a dataframe to make the information easier to interpret
sst_df = pd.DataFrame(sst_dict)

Now let's see what the dataframe looks like.

In [ ]:
sst_df

The data is easier to read and easier to manipulate when it is in the dataframe (table) format. 

The next step is to clean up the data before plotting. The `millis` property contains date and time information in a strange format so we will convert it to a format that is more intuitive and then create year, month, day, and day of the year (DOY) variables from it.

In [ ]:
# Function to add date variables to DataFrame.
# This function just uses pandas functions on the millis column to create date time columns

def add_date_info(df):
  df['Timestamp'] = pd.to_datetime(df['millis'], unit='ms')
  df['Year'] = pd.DatetimeIndex(df['Timestamp']).year
  df['Month'] = pd.DatetimeIndex(df['Timestamp']).month
  df['Day'] = pd.DatetimeIndex(df['Timestamp']).day
  df['DOY'] = pd.DatetimeIndex(df['Timestamp']).dayofyear
  return df  

In [ ]:
# apply the date info function to the dataframe
sst_df = add_date_info(sst_df)

# Rename the sst column SST and remove the unnecessary millis and system:index columns
sst_df = sst_df.rename(columns={
    'sst': 'SST'
}).drop(columns=['millis', 'system:index'])

Now let's see what our cleaned up dataframe looks like.

In [ ]:
sst_df

#### Plotting the Dataframe

The hard part is over. We now have our reduced (i.e. summarized) earth engine data in a table format. If you wanted to, you could export the table to a .csv and create a simple graph in excel. However, once you get used to it, charting in python is excellent and has tons of libraries and functions for creating beautiful custom graphs. Because we are working in a notebook, we can even make them interactive!

We did not filter our image collection by date so our dataframe `sst_df` contains all of the data included in the MODIS Ocean Color Dataset, i.e. the median SST values for our Area Of Interest (AOI) since July 4th 2002. This data product is not daily, but it still contains a lot of data for us to visualize.

Let's start by creating a scatterplot in matplotlib, which you may have used before. 

#### Matplotlib

In [ ]:
# import matplotlib
import matplotlib.pyplot as plt

# Set up the subplots. This is your base.
fig, ax = plt.subplots(figsize=(14, 6))

# Add scatter plot with blue points that are 20% opaque. Use DOY on the x, and SST on the y axis
ax.scatter(sst_df['DOY'], sst_df['SST'],
           c='black', alpha=0.2)

# Add some parameters.
ax.set_title('Lake Erie Surface Temperature Cycle', fontsize=16) # set the title
ax.set_xlabel('Date', fontsize=14) # set the x-axis label
ax.set_ylabel('Temperature [C]', fontsize=14) # set the y-axis label
ax.set_ylim(-4, 30) # set the y-axis range
ax.grid(lw=0.2) #set the opacity of the background grid to 20%


plt.show()

Our graph shows us about what we would expect for a lake in the northern hemisphere! There seem to be a couple of outliers, but for the most part, the lake surface temperature follows the same pattern for all of our data points.

Matplotlib is a standard that is very customizable (and has tons of support pages across the internet). However, there are other libraries that make it easier to create beautiful visualizations.

For this exercise we will use Altair, but if you're interested, try checking out seaborn as well!

#### Altair

The matplotlib chart we created is nice, but it is only presenting some of the information we have because it is not separating out the different years. Let's try another chart with Altair where we plot different years in different colours. 

In [ ]:
# Set the chart base up
base = alt.Chart(sst_df).encode(
    # set x-axis with DOY (see explanation of Q in comments below)
    # the scale specifies the range of our x-axis
    x=alt.X('DOY:Q', scale=alt.Scale(domain=[0, 370])),
    # set y-axis as SST and set range from -5 to 30
    y=alt.Y('SST:Q', scale=alt.Scale(domain=[-5, 30])),
    # Change the point colours by Year value using the viridis colour scheme
    color=alt.Color('Year:O', scale=alt.Scale(scheme='viridis')))

# add the points to the map as circles
points = base.mark_circle()

# set overall properties
(points).properties(width=600, height=350)

## Note: Altair uses Q and O to specify the kind of data
# Q = quantitative, or continuous data (like SST)
# O = ordinal, or categorical data (like Year)

This graph now helps us see which years the outliers are in. It seems like the outliers come from many different years -- there is no obvious trend of one year having higher or lower than normal SST.

As mentioned, Altair also allows for interactive charts. Now we can adapt the above code to make it interactive. We will also add lines that connect our points for each year to our chart.

In [ ]:
# Set the highlight parameters
# When we hover our mouse over a point, we want all the points that share that year
# to be highlighted
highlight = alt.selection(
    type='single', on='mouseover', fields=['Year'], nearest=True)

# Set the base, this is the same as above.
base = alt.Chart(sst_df).encode(
    x=alt.X('DOY:Q', scale=alt.Scale(domain=[0, 370])),
    y=alt.Y('SST:Q', scale=alt.Scale(domain=[-5, 30])),
    color=alt.Color('Year:O', scale=alt.Scale(scheme='turbo')))

# Add points - change the size based on the highlight condition we set above
points = base.mark_circle().encode(
    size=alt.condition(~highlight, alt.value(10), alt.value(15)),
    tooltip=[
        alt.Tooltip('Year:O', title='Year'),
        alt.Tooltip('DOY:Q', title='DOY'),
        alt.Tooltip('SST:Q', title='SST')
    ]).add_selection(highlight)

# Add lines - change the size based on the highlight condition
lines = base.mark_line().encode(
    size=alt.condition(~highlight, alt.value(1), alt.value(4)))

# Set the chart properties and add .interactive()
(points + lines).properties(width=600, height=350).interactive()

The above chart is interactive - when you hover over a value, it will give you information about the point. You can also zoom in on the graph and drag it up, down, right, and left.

Interactive maps and charts are a great way to make use of notebook functionality. 

#### Plot Chl-a Trend for 2015

For our last example, we will modify our code from above to look at the trend in chlorophyll concentration for 2015. There was a big algal bloom in 2015 in Lake Erie, which we should be able to see if we plot out the values. The code below follows the same structure that we used above for SST, but without all the comments so you see it in a more condensed format.



In [ ]:
##### SET UP IMAGE COLLECTION AND AOI #####

# Create a collection from the chlor_a (Chlorophyll-a concentration) band
chla = modisOceanColor.select(['chlor_a'])

# Define an area of interest (aoi)
aoi = ee.Geometry.Polygon([[-83.594,41.326],
                           [-78.782,41.326],
                           [-78.782,42.972],
                           [-83.594,42.972],
                           [-83.594,41.326]])


In [ ]:
##### REDUCE IMAGE COLL #####

# Create reducer function for mapping across image collection
reduce_chla = create_reduce_region_function(
    geometry=aoi, reducer=ee.Reducer.median())

# Map reducing function across image collection and convert it to feature collection
chla_stat_fc = ee.FeatureCollection(chla.map(reduce_chla))

# Clean up by removing null values
chla_stat_fc = chla_stat_fc.filter(
    ee.Filter.notNull(chla.first().bandNames()))

In [ ]:
##### CONVERT COLLECTION TO TABLE #####

# Convert ee feature collection to 'client-side' dictionary
chla_dict = fc_to_dict(chla_stat_fc).getInfo()

# Convert dictionary to dataframe and add date info
chla_df = pd.DataFrame(chla_dict)
chla_df = add_date_info(chla_df)

# Clean up data columns and remove unnecessary columns
chla_df = chla_df.drop(columns=['millis', 'system:index'])

In [ ]:
chla_df

So far, most of what we have done is the exact same as what we did for sea surface temperature above. We have just changed our band to be `chlor_a` instead of `sst` and we changed our variable names to reflect that change. 

Our dataframe contains all of the data since 2002 but we just want data for 2015. We ***could*** have filtered our image collection at the beginning, the same way we did in previous labs, which would have been less data intensive because it would have meant that we didn't need to convert values from all of the other years to a dictionary and a dataframe. However, we can also filter our dataframe, which gives us a bit more flexibility in case we want to plot other years. It also gives us the opportunity to use some pandas functions to filter the table/dataframe.

In [ ]:
# Filter dataframe using loc
# This line creates a new dataframe that only include rows from the
# original df that have values of 2015 in the Year column

chla2015_df = chla_df.loc[chla_df['Year'] == 2015]

In [ ]:
highlight = alt.selection(
    type='single', on='mouseover', fields=['chlor_a'], nearest=True)

base = alt.Chart(chla2015_df).encode(
    x=alt.X('DOY:Q', scale=alt.Scale(domain=[0, 370]),title = "Day of the Year"),
    y=alt.Y('chlor_a:Q', scale=alt.Scale(domain=[0,15]),title='Chl-a Concentration'),
    # use the goldgreen color scheme and set the variable as Q (continuous)
    color=alt.Color('chlor_a:Q', scale=alt.Scale(scheme='goldgreen'))) 

points = base.mark_circle().encode(
    size=alt.condition(~highlight, alt.value(100), alt.value(120)),
    tooltip=[
        alt.Tooltip('Year:O', title='Year'),
        alt.Tooltip('DOY:Q', title='DOY'),
        alt.Tooltip('chlor_a:Q', title='Chl-a')
    ]).add_selection(highlight)

(points).properties(width=600, height=350).interactive()

In the chart above, we didn't need to colour the points based on chl-a concentration because it doesn't add any additional information (we already see the Chl-a pattern based on their position on the chart). However, it can be a useful visualization tool to put the focus on the change of values. You can use a single colour by changing the above code for base to this:

```
base = alt.Chart(sst2015_df).encode(
    x=alt.X('DOY:Q', scale=alt.Scale(domain=[0, 370]),title = "Day of the Year"),
    y=alt.Y('chlor_a:Q', scale=alt.Scale(domain=[0,15]),title='Chl-a Concentration'),
    color=alt.value('green')) 
```

The chart above shows that there are some pretty high Chl-a values in the late summer. Let's check out what the surface reflectance and Chl-a values look like on a map!

In [ ]:
SR_Aug2015 = ee.ImageCollection('MODIS/006/MOD09GA')\
        .filterDate('2015-08-01', '2015-09-30')\
        .filterBounds(aoi)\
        .map(maskEmptyPixels)\
        .map(maskClouds)\
        .median()

SR_May2015 = ee.ImageCollection('MODIS/006/MOD09GA')\
        .filterDate('2015-04-01', '2015-05-31')\
        .filterBounds(aoi)\
        .map(maskEmptyPixels)\
        .map(maskClouds)\
        .median()


Chla_Aug2015 = ee.ImageCollection('NASA/OCEANDATA/MODIS-Aqua/L3SMI').select(['chlor_a'])\
        .filterDate('2015-08-01', '2015-09-30')\
        .filterBounds(aoi)\
        .median()

Chla_May2015 = ee.ImageCollection('NASA/OCEANDATA/MODIS-Aqua/L3SMI').select(['chlor_a'])\
        .filterDate('2015-04-01', '2015-05-31')\
        .filterBounds(aoi)\
        .median()

SRvis = {'bands': ['sur_refl_b01', 'sur_refl_b04', 'sur_refl_b03'],
     'gain': 0.07,
     'gamma': 1.7
    }

Chlavis = {'min':0,'max':16,'palette': 'blue,teal,green'};

Map = geemap.Map()
Map.addLayer(SR_May2015,SRvis,'May 2015 Median SR')
Map.addLayer(SR_Aug2015,SRvis,'Aug 2015 Median SR')
Map.addLayer(Chla_May2015,Chlavis,'May 2015 Median Chla')
Map.addLayer(Chla_Aug2015,Chlavis,'Aug 2015 Median Chla')
Map.addLayerControl()
Map.centerObject(aoi,8)
Map

We can definitely see a difference between the surface reflectances for April/May and August/September but the Chl-a values are very different! 

This is because Chl-a is calculated using more than just the RGB bands and thus shows more variation that we can discern just from true colour imagery. 

## Questions to Submit

[Total marks = 10]

Answer the following questions in the space provided. Feel free to add additional code and text cells as needed. Make sure to show all of your code.

**In all cases**, if you use code chunks from another source (not a bad thing), make sure to reference where you found them.



### Q1

Display the median 2018 SST value for the whole globe on a map. Use `'palette': 'navy,blue,turquoise'` in your visualization parameters. Make sure your minimum and maximum values make sense.

In [ ]:
## Show your code here

### Q2

Create a scatter plot showing the median SST over 2018 for an area of your choice anywhere in the world (over water, of course). You can either choose a point, or a rectangle to define your area. Hint: you can get latitudes and longitudes by clicking on a map in this notebook or in google maps.

In [ ]:
## Show your code here

### Q3

Now that you have used Google Earth Engine, what do you think about it? Did you find it useful and/or interesting? What are its strengths and weaknesses? Are there any things you have found particularly difficult? Are there things you have noticed that are particularly cool? Describe your experience in full sentences.

// Write your answer here